Tensorflow with keras
===


# 1. Import Library

In [ ]:
import numpy
import numpy as np
import pandas as pd
from sklearn import preprocessing
numpy.random.seed(10)

import matplotlib
import matplotlib.pyplot as plt

%matplotlib inline 

In [ ]:
# concatation of Kaggle data, train.csv and test.csv
df_train=pd.read_csv("data/Titanic/train.csv")
df_test=pd.read_csv("data/Titanic/test.csv")
df_concat=pd.concat([df_train,df_test])

In [ ]:
df_concat.head()

In [ ]:
df_concat=df_concat.sort_values(['Name'])

In [ ]:
df_concat.info()

To get more stories, we use the data from <a href="http://biostat.mc.vanderbilt.edu/wiki/pub/Main/DataSets/titanic3.xls">titanic3.xls</a>

In [ ]:
df = pd.read_excel("data/titanic3.xls")
#df = pd.read_csv("data/Titanic/train.csv")

In [ ]:
import qgrid

In [ ]:
qgrid.show_grid(df,show_toolbar=True)

Let the first column be 'survived':

In [ ]:
cols=['survived','name','pclass' ,'sex', 'age', 'sibsp',
      'parch', 'fare', 'embarked']
df=df[cols]

In [ ]:
#cdivede data into two parts, train and test. 
msk = numpy.random.rand(len(df)) < 0.8
train_df = df[msk]
test_df = df[~msk]

In [ ]:
print('total:',len(df),
      'train:',len(train_df),
      'test:',len(test_df))

Data Preprocessing
---
- drop `name` column
- fill empty `age` (`fare`) with mean value,
- convert `sex` property, 0 for Female, 1 for other
- duplicate and divide 'embarked' into 3 different columns according its features, 'C,Q,S',
  via  ```pd.get_dummies```.
- Convert df into ndarray
- divide label, index=0 i.e. `survived`, and Features, the others, indeces [1:],
- normalize the `Features`'s values within 0 and 1. 

In [ ]:
def PreprocessData(raw_df):
    # drop name feature
    df=raw_df.drop(['name'], axis=1)
    # replace the nan in "age","fare" with mean 
    age_mean = df['age'].mean()
    df['age'] = df['age'].fillna(age_mean)
    fare_mean = df['fare'].mean()
    df['fare'] = df['fare'].fillna(fare_mean)
    # convert "sex" into int, 0 for Female, 1 for male
    df['sex']= df['sex'].map({'female':0, 'male': 1}).astype(int)
    
    # divide "embarked" into individual features
    x_OneHot_df = pd.get_dummies(data=df,columns=["embarked" ])

    ndarray = x_OneHot_df.values
    Features = ndarray[:,1:]
    Label = ndarray[:,0]
    
    minmax_scale = preprocessing.MinMaxScaler(feature_range=(0, 1))
    scaledFeatures=minmax_scale.fit_transform(Features)    
    
    return scaledFeatures,Label

In [ ]:
train_Features,train_Label=PreprocessData(train_df)
test_Features,test_Label=PreprocessData(test_df)

In [ ]:
cols=['survived','name','pclass' ,'sex', 'age', 'sibsp',
      'parch', 'fare', 'embarked']

cols_name=['pclass','sex','age','sibsp','ParChild','fare','C','Q','S']

for i in range(9):
    print(cols_name[i],": ",train_Features[300][i])
        
#train_Features[0]

<big style="font-size:2em;color:yellow;background-color:black;">
    <font color="orange">**õ|o**</font> Question<big style="font-size:0.7em;color:brown;background-color:white;padding:-2pt"> 
about data prepocessing
</big>
</big>

---
Here, train and test data were preprocessed by the same function, "```PreprocessData()```". Is there any difference with the case of 1&deg;) preprocessing raw data first and 2&deg;) dividing into train and test data afterwards?

晉惠帝: 「<font color="red"> 何不食肉糜 ?</font>」

# 3. Create Model 
Artificial Neural Network:
<table>
   <tr>    
       <td><img src="imgs/neuron.png"  width=100% /></td>
       <td><img src="imgs/NN.jpg" width=90% /></td>
   </tr>
</table>    
The structure is desdigned as:
- 9-dimensional Input layer since 9 -features here,
- the first hideen layer is given in 40 cells,
- the second hideen layer is given in 30 cells,
- and 1-dimension output layer is given, one label (`suvived`) here.

**Reference**. <a href="#Note">Why Neural Network, deep learning</a>?

In [ ]:
def nn_graph(nodes,colors=[[1.0, 0.5, 0.5],[0.5, 0.5, 1],[0.5, 0.7509803921568627, 0.5]],fname=None):
    """
    Draw neural network graph with a given number of nodes per layer.
    
    --inputs:
    
    nodes: List with the number of nodes in each layer.
    colors (optional): List of colors for each layers' nodes, one color per layer.
    fname (optional): File name to save the figure.
    """
    
    nodes=[n for n in nodes if isinstance(n,int)]
    
    fig = plt.figure(figsize=(12,8))
    ax = fig.add_subplot(1, 1, 1)
    
    layers=len(nodes)
    mx=np.max(nodes)
    node_r=0.3
    
    plt.text(-0.5, 7, "Input", fontsize=16)
    plt.text(1.5, 7, "Hidden Layer ...", fontsize=16,color="grey")
    plt.text(5.5, 7, "Output", fontsize=16)
    
    plt.figure(figsize=(1.5*layers,mx))
    
    for i in range(layers):
        x=np.zeros(nodes[i])+2*i
        y=range(nodes[i])
        if (nodes[i]<mx):
            y+=(mx-nodes[i])/2
            
        #draw nodes
        for px,py in zip(x,y):
            circ = plt.Circle((px, py), radius=node_r, color=colors[i%len(colors)])
            ax.add_patch(circ)
        
        #draw connecting lines
        if i>0:
            for a in prevy:
                for b in y:
                    ax.plot([2*(i-1)+node_r,2*i-node_r],[a,b],'gray')
                    #ax.margins(0.2, 0.2)
        
        prevy=y
        
    
    ax.axis('off')
    ax.set_aspect("equal");
    
    if fname!=None:
        fig.savefig(fname,bbox_inches='tight',dpi=300)
        print('figure saved as: ',fname) 

In [ ]:
nodes=[5,7,6,1]
nn_graph(nodes,fname='nn.png')

In [ ]:
from keras.models import Sequential
from keras.layers import Dense,Dropout

activation functions
---
\begin{eqnarray}
\text{relu} &=& 0 \text{ if } x\le0\\
            &=& x \text{ if } x>0\\
\text{sigmod} &=& \frac{1}{1+e^{-x}}          
\end{eqnarray}

In [ ]:
import matplotlib.pylab as plt

%matplotlib inline


In [ ]:
def logit(z):
    return 1 / (1 + np.exp(-z))

In [ ]:
z = np.linspace(-5, 5, 200)

plt.plot([-5, 5], [0, 0], 'k-')
plt.plot([-5, 5], [1, 1], 'k--')
plt.plot([0, 0], [-0.2, 1.2], 'k-')
plt.plot([-5, 5], [-3/4, 7/4], 'g--')
plt.plot(z, logit(z), "b-", linewidth=2)
props = dict(facecolor='black', shrink=0.1)
plt.annotate('Saturating', xytext=(3.5, 0.7), xy=(5, 1), arrowprops=props, fontsize=14, ha="center")
plt.annotate('Saturating', xytext=(-3.5, 0.3), xy=(-5, 0), arrowprops=props, fontsize=14, ha="center")
plt.annotate('Linear', xytext=(2, 0.2), xy=(0, 0.5), arrowprops=props, fontsize=14, ha="center")
plt.grid(True)
plt.title("Sigmoid activation function", fontsize=14)
plt.axis([-5, 5, -0.2, 1.2]);


In [ ]:
def leaky_relu(z, alpha=0.01):
    return np.maximum(alpha*z, z)

In [ ]:
plt.plot(z, leaky_relu(z, 0.05), "b-", linewidth=2)
plt.plot([-5, 5], [0, 0], 'k-')
plt.plot([0, 0], [-0.5, 4.2], 'k-')
plt.grid(True)
props = dict(facecolor='black', shrink=0.1)
plt.annotate('Leak', xytext=(-3.5, 0.5), xy=(-5, -0.2), arrowprops=props, fontsize=14, ha="center")
plt.title("Leaky ReLU activation function", fontsize=14)
plt.axis([-5, 5, -0.5, 4.2]);


In the last few years, ```relu``` becomes most popular activation function than others: 

In [ ]:
model = Sequential()

In [ ]:
model.add(Dense(units=40, input_dim=9, 
                kernel_initializer='uniform', 
                activation='relu'))

In [ ]:
model.add(Dense(units=30, 
                kernel_initializer='uniform', 
                activation='relu'))

In [ ]:
model.add(Dense(units=1, 
                kernel_initializer='uniform',
                activation='sigmoid'))

# 4. Train model

```
survived
                        Y    F
Y'i labels (True Val):  0    1  
Yi   predict Value   : 0.2  0.8
```
binary_crossentropy = $\sum\limits_i -Y'_i\log Y_i= -0\cdot\log0.2- 1\cdot\log0.8\sim0.223$ 

Optimizer: optimize the loss function by continuously updating weights and biase, i.e stochastic gradient decent (SGD) is the famous one, general optitions, sgd, rmsprop, adam etc; the last one becomes popular recently. 

Define the model and train it:
```python
model.compile(loss='binary_crossentropy',
              optimizer='adam', metrics=['accuracy'])

train_history =model.fit(x=Features, y=Label, validation_split=0.1, 
                         epochs=30, batch_size=30,verbose=0)
```
where the parameters are defined as follows:
- loss: Loss function to minimize, include mse;
- split=0.1: 10% data for validation;
- batch_size=30: 30 data trained concurrenct;
- epochs=30: 30 tarining steps;
- verbose=0: disable trainging result and 2 for enable.

In [ ]:
def creat_sgd_model():
    sgd = SGD(lr=0.01, momentum=0.8, decay=0.0, nesterov=False)
    model.compile(loss='binary_crossentropy', optimizer=sgd, metrics=['accuracy'])
    return model

In [ ]:
sgd_model = creat_adam_model()
sgd_train_history = sgd_model.fit(x=train_Features, y=train_Label, 
                         validation_split=0.1, epochs=30, batch_size=30,verbose=0)

sgd_model.evaluate(x=test_Features,  y=test_Label)[1]

In [ ]:
def creat_adam_model():
    model.compile(loss='binary_crossentropy',optimizer='adam', metrics=['accuracy'])
    return model

In [ ]:
adam_model = creat_adam_model()
adam_train_history = adam_model.fit(x=train_Features, y=train_Label, 
                         validation_split=0.1, epochs=30, batch_size=30,verbose=0)

adam_model.evaluate(x=test_Features,  y=test_Label)[1]

# 6. Print History
How good id the fit of model? The visualization of accuracy and loss should explains the result. 

In [ ]:
import matplotlib.pyplot as plt
def show_train_history(train_history,train,validation):
    plt.plot(train_history.history[train])
    plt.plot(train_history.history[validation])
    plt.title('Train History')
    plt.ylabel(train)
    plt.xlabel('Epoch')
    plt.legend(['train', 'validation'], loc='upper left')
    plt.show()

Whatever train or validation result, accuracy had increased about 0.75 after 5 epoch, and loss related had also decreasing fast under 0.5 after 5 epoch.

In [ ]:
show_train_history(train_history,'acc','val_acc')

In [ ]:
show_train_history(train_history,'acc','val_acc')

In [ ]:
show_train_history(train_history,'loss','val_loss')

Visualization by Plotly
---
Benifited from the HTML library, **d3.js**, plotly provides more awesome visualization tool for data display.  The following demo shows how to make an off-line data picture:

In [ ]:
import plotly.graph_objs as go

import plotly
from plotly.offline import init_notebook_mode,iplot
init_notebook_mode()

In [ ]:
def show_train_history_plotly(train_history,train,validation):
    trace0 = go.Scatter(
     # x=np.arange(len(train_history.history['train])),
     y=train_history.history[train],
     name=train   
    )
    trace1 = go.Scatter(
     # x=np.arange(len(train_history.history['train])),
     y=train_history.history[validation],
     name=validation   
    )
    
    data = [trace0,trace1]
    fig = go.Figure(data=data)
    iplot(fig)

In [ ]:
np.arange(len(train_history.history['loss']))

In [ ]:
show_train_history_plotly(train_history,'loss','val_loss')

# Evaluation of Model Accuracy

In [ ]:
scores = model.evaluate(x=test_Features, 
                        y=test_Label)

In [ ]:
scores[1]

Save/Load Training Model
---
1.  Use the pickle operation to serialize your machine learning algorithms and save the serialized format to a file;  later we can load this file to deserialize your model and use it to make new predictions.
- Joblib is part of the SciPy ecosystem and provides utilities for pipelining Python jobs.
It provides utilities for saving and loading Python objects that make use of NumPy data structures, efficiently. This can be useful for some machine learning algorithms that require a lot of parameters or store the entire dataset (like K-Nearest Neighbors).

In [ ]:
import pickle
filename = 'finalized_model.sav'
pickle.dump(model, open(filename, 'wb'))
 
# some time later...
 
# load the model from disk
loaded_model = pickle.load(open(filename, 'rb'))
#result = loaded_model.score(X_test, Y_test)
#print(result)

Save/Load Keras Model
---
Keras separates the concerns of saving your model architecture and saving your model weights.

Model weights are saved to HDF5 format. This is a grid format that is ideal for storing multi-dimensional arrays of numbers.

The model structure can be described and saved using two different formats: JSON and YAML:
1. JSON: Keras provides the ability to describe any model using JSON format with a to_json() function. This can be saved to file and later loaded via the model_from_json() function that will create a new model from the JSON specification, and the weights are saved directly from the model using the save_weights() function and later loaded using the symmetrical load_weights() function.
- YAML: the model is described using YAML, saved to file model.yaml and later loaded into a new model via the model_from_yaml() function. Weights are handled in the same way as above in HDF5 format as model.h5.



In [ ]:
model

In [ ]:
# serialize model to JSON
model_json = model.to_json()
with open("model.json", "w") as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
model.save_weights("model.h5")
print("Saved model to disk")

In [ ]:
from keras.models import model_from_json

# load json and create model
json_file = open('model.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
loaded_model = model_from_json(loaded_model_json)
# load weights into new model
loaded_model.load_weights("model.h5")
print("Loaded model from disk")

In [ ]:
loaded_model.compile(loss='binary_crossentropy', 
              optimizer='adam', metrics=['accuracy'])
scores1 = loaded_model.evaluate(x=test_Features, 
                        y=test_Label)
scores1[1]

In [ ]:
from keras.models import model_from_yaml

model_yaml = model.to_yaml()
with open("model.yaml", "w") as yaml_file:
    yaml_file.write(model_yaml)
# serialize weights to HDF5
model.save_weights("model2.h5")
print("Saved model to disk")

In [ ]:
from keras.models import model_from_yaml

# load YAML and create model
yaml_file = open('model.yaml', 'r')
loaded_model_yaml = yaml_file.read()
yaml_file.close()
loaded_model2 = model_from_yaml(loaded_model_yaml)
# load weights into new model
loaded_model2.load_weights("model2.h5")
print("Loaded model from disk")

In [ ]:
loaded_model2.compile(loss='binary_crossentropy', 
              optimizer='adam', metrics=['accuracy'])
scores2 = loaded_model2.evaluate(x=test_Features, 
                        y=test_Label)
scores2[1]

Note
---
Why is Neural Network concered with "XOR":

```
   Pred\Fact    True   False
    
      True        T      F
     
      Flase       F      T
```
We can not divide the output by single line, (least square, for instance); thus, introduce additional conditions to help to make decision, (i.e. insert some "<font color="brown">hidden layer</font>", or think that we use more lines to make final output).  

In [ ]:
def cm_plot(y, yp):
  cm = confusion_matrix(y, yp) 
  
  plt.matshow(cm, cmap=plt.cm.Oranges) 
  plt.colorbar() 
  
  for x in range(len(cm)): 
    for y in range(len(cm)):
      plt.annotate(cm[x,y], xy=(x, y), horizontalalignment='center', verticalalignment='center',
                   size=16,color='lightblue')
  
  plt.ylabel('True label',size=16) 
  plt.xlabel('Predicted label', size=16) 
  #return plt

In [ ]:
train_Label

In [ ]:
yp=model.predict_classes(train_Features).reshape(len(train_Label))

In [ ]:
cm_plot(train_Label,yp)

# Prediction

# Additional data of Jack and Rose

In [ ]:
Jack = pd.Series([0 ,'Jack',3, 'male'  , 23, 1, 0,  5.0000,'S'])
Rose = pd.Series([1 ,'Rose',1, 'female', 20, 1, 0, 100.0000,'S'])

In [ ]:
JR_df = pd.DataFrame([list(Jack),list(Rose)],  
                  columns=['survived', 'name','pclass', 'sex', 
                   'age', 'sibsp','parch', 'fare','embarked'])

In [ ]:
all_df=pd.concat([df,JR_df])

In [ ]:
all_df[-2:]

In [ ]:
all_df[-2:]

# Survived Prediction 

In [ ]:
all_Features,Label=PreprocessData(all_df)

In [ ]:
all_probability=model.predict(all_Features)

In [ ]:
all_probability[:10]

In [ ]:
pd=all_df
pd.insert(len(all_df.columns),
          'probability',all_probability)

# Were Jack and Rose survived

In [ ]:
pd[-2:]

# Could be survived but not

In [ ]:
pd[(pd['survived']==0) &  (pd['probability']>0.9) ]

In [ ]:
pd[:6]

In [ ]:
pd[73:74]

In [ ]:
pd[285:290]

Stories
---
1. Guests number[1,2,3,4,73], Allison family, are the family and traveled with their maid. Allison family didn't know the baby, Trevor, **member 1**, had been aboard lifeboard with their maid, Alice, **member 73**, and stayed on Titanic. Thus they, **members [2,3,4]** sank with Titanic, so sad.
2. Guest number 69, Lady Ann, could not escape herself  without her great dane (dog), and stay with her pet; therefore became one of dead in this wreck.
3. Guests number [286,287], Mr. and Mrs. Straus, cofounder of Merci's New York, were also dead in this wreck thought Mrs. Straus had chance to be survived; but she refused to leave her husband and be aboard lifeboard alone. "Where you go, I go", others told what Mrs. Straus said at last. 